In [1]:
import whobpyt
from whobpyt.datatypes import par, Recording
from whobpyt.models.RWW import RNNRWW, ParamsRWW
from whobpyt.models.RWWEI2 import RWWEI2, ParamsRWWEI2
from whobpyt.optimization.custom_cost_RWW import CostsRWW
from whobpyt.run import Model_fitting

import os
import sys 

# array and pd stuff
import numpy as np
import pandas as pd
from scipy.io import loadmat # for reading in the .mat files

import torch

# viz stuff
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'whobpyt'

In [24]:
print("Is cuda avaliable?")
print(torch.cuda.is_available())

device = torch.device("cpu") #Options: "cpu" or "cuda"

Is cuda avaliable?
True


In [3]:
# Normalise the FC data and find Correlation matrix
def normalise_correlate_fc(fc):
    fc_emp = fc / np.max(fc)
    fc_emp = np.corrcoef(fc_emp.T)
    return fc_emp

In [4]:
def get_avg_fc(fcs):
    corr_matrices = []

    for data in fcs:
        fc = normalise_correlate_fc(data)
        corr_matrices.append(fc)

    stacked_corr = np.stack(corr_matrices, axis = 0)
    avg_corr = np.mean(stacked_corr, axis =0)
    
    return avg_corr

In [7]:
# load raw data and get SC empirical BOLD and FC

sc = np.genfromtxt('data/DTI_fiber_consensus_HCP.csv', delimiter=',')
with open('data/healthy_subjs.txt', 'r') as file:
    # Read all lines into a list
    fc_files = file.readlines()

fc_files = [file.strip() for file in fc_files]

fcs = []

for f in fc_files[:10]:
#     print(f)
    fc = loadmat(f)
    fc = fc['ROI_activity'][:100, :]
    fc = fc.T
    # print(fc.shape)
    fcs.append(fc)

print(len(fcs))

10


In [29]:
# define options for wong-wang model
node_size = 100
mask = np.tril_indices(node_size, -1)
num_epochs = 5
TPperWindow = 20
step_size = 0.05
input_size = 2
tr = 0.75
repeat_size = 5

In [10]:
SC = (sc + sc.T) * 0.5
sc = np.log1p(SC) / np.linalg.norm(np.log1p(SC))
print(sc.shape)

fc_emp = get_avg_fc(fcs)

# normalise all fcs
for fc in fcs:
    fc = fc / np.max(fc)
    
# prepare data structure of the model
fMRIstep = tr

avgfc = Recording(fc_emp.T, fMRIstep)

# for multiple empirical bolds:
rcds = []

for f in fcs:
    rcds.append(Recording(f.T, fMRIstep))
print(len(rcds))

(100, 100)
10


In [31]:
# get model parameters structure and define the fitted parameters by setting non-zero variance for the model

# params = ParamsRWWEI2(G=par(100, 100, 1/np.sqrt(10), True, True), g_EE=par(3.5, 3.5, 1/np.sqrt(50), True, True), g_EI =par(0.42, 0.42, 1/np.sqrt(50), True, True), \
#                    g_IE=par(0.42, 0.42, 1/np.sqrt(50), True, True), I_0 =par(0.2), std_in=par(0.0), std_out=par(0.00))
# params.to(device)

paramsNode = ParamsRWWEI2(num_regions=100)
paramsNode.G = par(torch.tensor(2.0), fit_par = True, asLog = True)
paramsNode.to(device)

# call model want to fit
# model = RNNRWW(node_size, TPperWindow, step_size, repeat_size, tr, sc, True, params)
model = RWWEI2(num_regions=100, params=paramsNode, Con_Mtx=sc, step_size=step_size, sim_len=tr*1000, device = device)

# %%
# create objective function
ObjFun = CostsRWW(model, device=device)

# %%
# call model fit
F = Model_fitting(model, ObjFun, device=device)

/rds/general/user/dc420/home/ad_modelling_fyp/whobpyt/datatypes/parameter.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.val = torch.tensor(val, dtype=torch.float32).to(device)


TypeError: RWWEI2.__init__() missing 1 required positional argument: 'Dist_Mtx'

In [30]:
# Model Training
# ---------------------------------------------------
#
F.train(u = 0, empRecs = rcds, num_epochs = num_epochs, TPperWindow = TPperWindow, learningrate = 0.05, )

Epoch:  0
epoch:  0 loss: 8.616871 Pseudo FC_cor:  0.05864909609208868 cos_sim:  -0.9961187318648489
epoch:  0 loss: 7.3086824 Pseudo FC_cor:  0.0025978414339729336 cos_sim:  -0.9462153362665393
epoch:  0 loss: 6.200033 Pseudo FC_cor:  0.06882899599707246 cos_sim:  -0.8523910932426988
epoch:  0 loss: 6.1356406 Pseudo FC_cor:  0.032581616316987784 cos_sim:  -0.995627969452897
epoch:  0 loss: 5.0131907 Pseudo FC_cor:  0.036970052779766345 cos_sim:  -0.9980271132105848
epoch:  0 loss: 5.508816 Pseudo FC_cor:  0.08180350799982895 cos_sim:  -0.9990860530006043
epoch:  0 loss: 5.1826406 Pseudo FC_cor:  -0.004234801622686044 cos_sim:  0.6716482492087398
epoch:  0 loss: 5.113417 Pseudo FC_cor:  0.08138770590581489 cos_sim:  -0.8804539205630416
epoch:  0 loss: 4.7641306 Pseudo FC_cor:  0.04742515676753192 cos_sim:  -0.9958665093837051
epoch:  0 loss: 4.466594 Pseudo FC_cor:  -0.007319240608141967 cos_sim:  -0.9981164314755796
Epoch:  1
epoch:  1 loss: 4.569216 Pseudo FC_cor:  0.1846213172702461

KeyboardInterrupt: 

In [ ]:
# Plots of loss over Training
plt.plot(np.arange(1,len(F.trainingStats.loss)+1), F.trainingStats.loss, label='loss')
plt.title("Total Loss over Training Epochs")
plt.show()

# %%
# Plots of parameters values over Training
plt.plot(F.trainingStats.fit_params['g_EE'], label = "g_EE")
plt.plot(F.trainingStats.fit_params['g_EI'], label = "g_EI")
plt.plot(F.trainingStats.fit_params['g_IE'], label = "g_IE")
plt.legend()
plt.title("Local Coupling Variables Changing Over Training Epochs")
plt.show()

In [ ]:
F.evaluate(u = 0, empRecs = rcds[0], TPperWindow = TPperWindow, base_window_num = 20)

In [ ]:
# Plot E I and simulated BOLD
fig, ax = plt.subplots(1, 3, figsize=(12, 8))
ax[0].plot(F.lastRec['E'].npTS().T)
ax[0].set_title('Test: E')
ax[1].plot(F.lastRec['I'].npTS().T)
ax[1].set_title('Test: I')
ax[2].plot(F.lastRec['bold'].npTS().T)
ax[2].set_title('Test: BOLD')
plt.show()

In [ ]:
# %%
# Plot the FC and the test FC
fig, ax = plt.subplots(1, 2, figsize=(14, 8))
im0 = ax[0].imshow(fc_emp, cmap='bwr')
ax[0].set_title('The empirical FC')
fig.colorbar(im0, ax=ax[0], fraction=0.046, pad=0.04, label='Connection Weights')
test = F.lastRec['bold'].npTS()

fc_test = np.corrcoef(F.lastRec['bold'].npTS())
im1 = ax[1].imshow(fc_test, cmap='bwr')
ax[1].set_title('The simulated FC')
fig.colorbar(im1, ax=ax[1], fraction=0.046, pad=0.04, label='Connection Weights')
plt.show()